## Librerie

In [1]:
from scipy.optimize import minimize, basinhopping
import numpy as np
import pandas as pd

## Parametri problema

In [2]:
# Consumi famiglia 1, famiglia 2, ...
consumi = np.array([1500, 5700, 3400, 300])

# Produzioni ricetta1, ricetta2, ...
produzioni = np.array([3000, 6300, 1200])

# Ricette
#           | Ricetta1 | Ricetta2 | ...
# --------------------------------------
# Famiglia1 |          |          |
# Famiglia2 |          |          |
# ...
ricette = np.array([
    0.25, 0.2, 0.3,
    0.43, 0.5, 0.35,
    0.3, 0.27, 0.35,
    0.02, 0.03,  0
])

# Composizioni ricette per famiglia
#           | Materiale1 | Materiale2 | ...
# --------------------------------------
# Famiglia1 |            |            |
# Famiglia2 |            |            |
# ...
composizioni_famiglia = np.array([
    0.58, 0.42, 0,
    1, 0, 0,
    0, 1, 0,
    0, 0, 1
])

# Range ammissibile percentuale materiale per ricetta
#           | Materiale1        | Materiale2 | ...
# ------------------------------------------------
# Ricetta1  | (val att. ,range) |            |
# Ricetta2  |                   |            |
# ...
range_ric_mat = np.array([
    [(0.58, 0.01), (0.396, 0.003), (0.024, 0.001)], # ricetta 0
    [(0.625, 0.005), (None, None), (None, None)],
    [(0.62, 0.01), (None, None), (None, None)],
])

In [3]:
%run ./importazione.ipynb

(10,)
(28,)
(28, 10)
(28, 13)
(10, 13)
(10, 13)
(10, 13)


In [4]:
print('Verifica dimensioni matrici')
print(f'Consumi (num. famiglie)): {consumi.shape}')
print(f'Produzioni (num. ricette): {produzioni.shape}')
print(f'Ricette (famiglie x ricette): {ricette.shape}')
print(f'Composizioni (famiglie x materiali): {composizioni_famiglia.shape}')
print(f'Range (ricette x materiali): {range_ric_mat.shape}')

Verifica dimensioni matrici
Consumi (num. famiglie)): (28,)
Produzioni (num. ricette): (10,)
Ricette (famiglie x ricette): (280,)
Composizioni (famiglie x materiali): (364,)
Range (ricette x materiali): (10, 13)


## Accrocchio per compattare ricetta

In [5]:
POSITIONS_COMP = []
RICETTA_COMP = []

for index, value in enumerate(ricette):
    if value != 0:
        POSITIONS_COMP.append(index)
        RICETTA_COMP.append(value)

def rebuild_ricetta(ricetta_comp):
    rebuild = np.zeros(len(ricette))
    for index, pos in enumerate(POSITIONS_COMP):
        rebuild[pos] = ricetta_comp[index]
    return rebuild

print(rebuild_ricetta(RICETTA_COMP)[0:20])
print(ricette[0:20])
print(len(RICETTA_COMP))

[0.    0.029 0.    0.    0.    0.    0.    0.    0.    0.    0.455 0.48
 0.03  0.475 0.    0.    0.    0.    0.005 0.   ]
[0.    0.029 0.    0.    0.    0.    0.    0.    0.    0.    0.455 0.48
 0.03  0.475 0.    0.    0.    0.    0.005 0.   ]
76


## Calcolo resa globale

In [6]:
tot_consumi = np.sum(consumi)
tot_produzioni = np.sum(produzioni)
resa_globale = tot_consumi / tot_produzioni
f'{resa_globale=}'

'resa_globale=np.float64(1.0389346559077535)'

## Funzioni di calcolo

In [7]:
# Calcola matrice consumi moltiplicando matrice ricetta in input per produzioni
def calc_mat_consumi(ricetta):
    # ricetta = rebuild_ricetta(ricetta) #REBUILD
    return ricetta.reshape(-1, len(produzioni)) * produzioni #/ resa_globale

In [8]:
# Calcola vettore consumi complessivi partendo da produzioni iniziali e matrice consumi
def calc_tot_consumi(matrice_consumi):
    return np.sum(matrice_consumi, axis=1)

In [9]:
# Calcolo errore su totali consumi
def calc_err_totali(ricetta):
    matrice_consumi = calc_mat_consumi(ricetta)
    tot_consumi = calc_tot_consumi(matrice_consumi)
    tot_err = np.sqrt(np.sum(np.square(tot_consumi-consumi)))/1000
    return tot_err

In [10]:
# Calcola rese per famiglia 
# (consumi per famiglia / produzione)
def calc_tot_resa(matrice_consumi):
    return np.sum(matrice_consumi, axis=0)/produzioni

In [11]:
# Calcolo errore su percentuali prod. effettive rispetto a resa totale
def calc_error_resa(ricetta):
    matrice_consumi = calc_mat_consumi(ricetta)
    tot_resa = calc_tot_resa(matrice_consumi)
    return np.sum(np.square(tot_resa - resa_globale))

In [12]:
# Percentuali di ciascun materiale in una ricetta
def perc_mat(ricetta, id_ric):
    cons_fam = calc_mat_consumi(ricetta)
    compos = composizioni_famiglia.reshape(len(consumi), -1)
    cons_ricetta0 = np.vstack(cons_fam[:,id_ric])
    cons_materiali_ricetta0 = cons_ricetta0 * compos
    tot_cons_ricetta0 = np.sum(cons_materiali_ricetta0, axis=0)
    tot_perc_ricetta0 = tot_cons_ricetta0 / np.sum(tot_cons_ricetta0) if np.sum(tot_cons_ricetta0) != 0 else np.zeros((len(tot_cons_ricetta0),1))
    return tot_perc_ricetta0

In [13]:
# Funzione errore percentuale materiale (obiettivo: >=0)
def err_perc_mat(ricetta, id_ric, id_mat, expected_val, expected_error):
    return expected_error - np.abs(perc_mat(ricetta, id_ric=id_ric)[id_mat] - expected_val)

## Ottimizzazione resa e consumi totali

In [14]:
%%time

constraints = [
    {'type': 'eq', 'fun': calc_err_totali},
]

# bounds = list(( (0, 1) for x in range(len(RICETTA_COMP) ))) # solo se RICETTA_COMP
bounds = list(( (0, 1) for x in range(len(ricette) )))

count_iterations = 0
def callback(x, y=None):
    global count_iterations
    print(f'{count_iterations} - {calc_err_totali(x)}')
    count_iterations += 1

# Minimizzazione con TRUST-CONSTR
res = minimize(
    calc_error_resa, 
    # RICETTA_COMP, 
    ricette,
    method='SLSQP',
    constraints=constraints,
    bounds=bounds,
    callback=callback,
    options={'disp': True, 'maxiter':100}
)


0 - 98255.15238461975
1 - 3923.6961888487435
2 - 6372.73080198374
3 - 9979.397852476332
4 - 6051.470165313267
5 - 5106.856406812529
6 - 7644.159434935086
7 - 6219.257164863578
8 - 7423.741121555668
9 - 5293.798394109715
10 - 8213.322510567303
11 - 3219.092245854024
12 - 2794.0791587909266
13 - 4443.8601390845415
14 - 6987.99877122175
15 - 2285.3028015379778
16 - 2960.7144893855857
17 - 2629.6509234157506
18 - 3405.306108011548
19 - 1921.9620286776146
20 - 2136.446915577483
21 - 2398.3967240422367
22 - 1846.0039123245097
23 - 3132.688361584387
24 - 1915.435956691525
25 - 3560.554928248685
26 - 3856.0356997015056
27 - 2136.732562801342
28 - 5019.631192276814
29 - 2197.403540438962
30 - 4877.373816910804
31 - 1647.5961508833543
32 - 1736.9049133210178
33 - 1590.9204972007524
34 - 733.7665927625197
35 - 3770.493264060573
36 - 2336.3039271845196
37 - 854.4308091256071
38 - 1102.881230150193
39 - 847.1207533917252
40 - 1050.9163924938334
41 - 520.73728216331
42 - 378.6881526768173
43 - 319.5

In [15]:
# Esperimento
# Proviamo a escludere un constraint alla volta e vediamo se troviamo una situazione
# in cui il problema converge.

# for i in range(1, len(constraints)):
    
#     print(i)
    
#     const_red = constraints[0:i] + constraints[i+1:]

#     res = minimize(
#         calc_error_resa, 
#         ricette, 
#         method='SLSQP',
#         constraints=const_red,
#         bounds=bounds,
#         options={'disp': False, 'maxiter':100}
#     )

#     print(f'{res.success} - {res.message} ({res.nit} iterations)')


## Verifiche

In [16]:
pd.options.display.float_format = '{:.2f}'.format

In [17]:
# res.x = res.x / resa_globale
result = rebuild_ricetta(res.x)

In [18]:
print("Percentuali aggiustate (in %)")
print(np.sum(result.reshape(len(consumi), len(produzioni))*100, axis=0))
pd.DataFrame(
    result.reshape(len(consumi), len(produzioni))*100
)

Percentuali aggiustate (in %)
[12.28018827 44.67736674  8.81008806 12.28553549 32.48265871 54.35364477
  6.97776998 63.64675572 75.79765609 12.59197123]


,0,1,2,3,4,5,6,7,8,9
0,0.00,0.14,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.48,0.15,0.74,2.66,0.00,0.00,0.00,0.00,4.17,0.00
2,0.00,0.00,0.86,0.00,2.52,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,3.45,0.00,0.00,0.00,0.91,0.00
4,0.00,0.00,0.00,0.00,0.00,47.70,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,55.82,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,9.04
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,54.11,0.00
8,9.44,5.56,0.99,4.61,0.00,0.00,0.00,0.00,13.02,0.00
9,0.00,0.00,1.23,0.00,0.14,0.76,0.00,0.00,0.00,0.00


In [19]:
print("Matrice consumi")
pd.DataFrame(
    calc_mat_consumi(res.x)
)

Matrice consumi


,0,1,2,3,4,5,6,7,8,9
0,8340.36,167548.37,11557.31,686605.94,99635.55,56635.28,1721.04,39585.72,173153.41,2598.43
1,2765869.13,19532500.01,676059.83,50456348.77,354312.55,75467.78,1986.25,72322.59,652907.32,3512.86
2,8099.19,267696.73,1911664.69,645265.71,198833.20,46192.80,1496.98,25193.72,41346.14,2138.29
3,25263.68,91653.08,73193.84,837883.44,180046.24,54599.13,1232.59,29853.11,312332.85,1588.07
4,539.20,13168.78,258.98,53024.87,31.79,2710.84,3.49,60.74,32.18,432.06
5,2227.64,13806.98,2420.54,62632.58,3582.70,36159.09,1267.68,588586.01,5904.61,1688.92
6,235.53,1839.22,1609.07,10737.46,2689.87,1312.95,38.65,395.47,4413.63,354.72
7,2351.28,31129.82,5209.98,140897.21,2912.05,37011.86,1286.65,12236.00,101457.45,1726.62
8,1625756.58,9213618.61,660251.25,28885399.52,350336.35,75742.66,1993.52,71626.17,645807.94,3526.40
9,8523.19,88745.95,1578059.46,302802.80,165059.98,54605.95,1663.37,36315.89,156612.24,2499.35


In [20]:
print('Verifica totale consumi')
pd.DataFrame(
    [calc_tot_consumi(calc_mat_consumi(res.x))]
)

Verifica totale consumi


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,1247381.40,74591287.10,3147927.46,1607646.04,70262.94,718276.75,23626.57,336218.93,41534059.01,2394888.19,...,10410206.35,1486226.14,57881.34,2349.52,4441.68,12204.32,24326.55,212.26,6395.02,10842.42


In [21]:
print('Verifica errore consumi')
pd.DataFrame([calc_tot_consumi(calc_mat_consumi(res.x)) - consumi])

Verifica errore consumi


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,-0.60,-0.63,-0.42,-0.56,-0.56,-0.55,-0.43,-1.07,-0.33,-0.81,...,-0.65,-0.86,-0.66,-0.48,-0.32,-0.68,-0.45,-0.74,-0.48,-0.58


In [22]:
# np.sum(calc_mat_consumi(res.x)*resa_globale, axis=1) / consumi * 100

In [23]:
print(f'Verifica rese (resa globale: {resa_globale:.2f})')
pd.DataFrame(
    [calc_tot_resa(calc_mat_consumi(res.x))]
)

Verifica rese (resa globale: 1.04)


,0,1,2,3,4,5,6,7,8,9
0,1.04,1.04,1.04,1.04,1.04,1.04,1.04,1.04,1.04,1.04


In [24]:
print(f'Verifica errore rese (resa globale: {resa_globale:.2f})')
pd.DataFrame(
    [calc_tot_resa(calc_mat_consumi(res.x)) - resa_globale]
)

Verifica errore rese (resa globale: 1.04)


,0,1,2,3,4,5,6,7,8,9
0,0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00


### Errori percentuali materiali

In [27]:
def print_err_mat(id_ricetta, ricetta):
    return pd.DataFrame([
        perc_mat(ricette, id_ricetta) * 100,
        map(lambda x: x[0]*100, list(range_ric_mat[id_ricetta])),
        perc_mat(ricetta, id_ricetta) * 100
    ])

In [28]:
print('Percentuali materiali ricetta 0')
print_err_mat(0, res.x)

Percentuali materiali ricetta 0


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,55.65,13.37,0.12,0.14,0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,30.67
1,59.40,2.05,0.16,0.15,0.05,0.02,0.01,0.00,0.01,0.01,0.00,0.00,38.13
2,60.25,5.01,0.13,0.14,0.05,0.00,0.00,1.03,0.02,0.00,0.00,0.00,33.38


In [29]:
print('Percentuali materiali ricetta 1')
print_err_mat(1, res.x)

Percentuali materiali ricetta 1


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,58.11,11.74,0.14,0.16,0.06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,29.79
1,58.00,2.90,0.16,0.16,0.10,0.02,0.01,0.00,0.01,0.01,0.00,0.00,38.62
2,59.59,3.11,0.15,0.16,0.05,0.00,0.00,0.57,0.01,0.00,0.00,0.00,36.37


In [30]:
print('Percentuali materiali ricetta 2')
print_err_mat(2, res.x)

Percentuali materiali ricetta 2


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,64.22,0.81,0.08,0.07,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,34.79
1,61.80,2.00,0.06,0.06,0.05,0.02,0.01,0.00,0.10,0.01,0.00,0.00,35.88
2,61.44,2.67,0.08,0.08,0.02,0.00,0.00,0.74,0.02,0.00,0.00,0.00,34.95


# xxx

In [31]:
# Scegliere un caso specifico (ricetta N, materiale M)

In [35]:
# Applicare algoritmo su caso specifico (P.65 documento)

In [33]:
# Valutare (con funzioni copiate da sopra) consumi totali e rese

In [34]:
# Valutare (con funzioni copiate da sopra) percentuali materiale M in ricetta N (e poi altri materiali/ricette per vedere se/come sono cambiati)